# Python lib dependencies

In [1]:
%pip install -q transformers accelerate


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# One SRT

In [2]:
srt_content = '''
1
00:00:00,000 --> 00:00:21,000
You just said something that's like very important can't be dogmatic when you're talking about vaccines or about anything.

2
00:00:21,000 --> 00:00:29,000
Yes it is good to keep an open mind isn't it and be flexible and look at a 360 degree view of things rather than your tunnel vision

3
00:00:29,000 --> 00:00:31,000
and what you're indoctrinated into isn't it?

4
00:00:31,000 --> 00:00:38,000
Yeah and especially if you know that indoctrination has been on purpose and profitable.

5
00:00:38,000 --> 00:00:45,000
One of the great things about your book is first of all your book is called dissolving illusions.

6
00:00:45,000 --> 00:00:55,000
I know I've talked about on the podcast a bunch of times but you also highlight a lot of things that we know are beneficial

7
00:00:55,000 --> 00:01:01,000
that somehow or another get lumped into nonsense like even cinnamon.

1
00:00:00,000 --> 00:00:07,000
 the joe rogan experience train by day joe rogan podcast by night all day

2
00:00:07,000 --> 00:00:17,000
 what's up what's up man good to see you good to see you uh so i guess we just get right into it

3
00:00:17,000 --> 00:00:26,000
 the last case that we talked about we had a very unfortunate incident happen after the podcast

4
00:00:26,000 --> 00:00:34,000
 about a month later yeah uh the gentleman beheaded somebody allegedly yes

5
00:00:34,000 --> 00:00:40,000
 allegedly there's a lot of allegedly's but yeah there's so many crazy things that case

6
00:00:40,000 --> 00:00:44,000
 the craziest thing was him uh trying to fool the security cameras with a wig

7
00:00:44,000 --> 00:00:50,000
 like i guess he didn't know how high resolution cameras had gotten over the 25 years that he was

8
00:00:50,000 --> 00:00:56,000
 in jail i mean apparently there's a lot he didn't know the only reason

9
00:00:56,000 --> 00:01:04,000
 i say allegedly is because um i'd be a bit of a hypocrite if i started calling him guilty um

10
00:01:04,000 --> 00:01:09,000
 before he has a trial but of course based on the surveillance um it doesn't look good
'''

with open('demo.srt','w') as srt:
  srt.write(srt_content)

#  加载模型

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "microsoft/phi-3-mini-128k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

print("模型加载完成！")

/home/caesar/Documents/code-projects/routine-utilities/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.66s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


模型加载完成！


# 提取字幕

In [4]:
import re

def extract_texts_from_srt(path):
    texts = []
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    for line in lines:
        line = line.strip()
        if re.match(r'^\d+$', line) or '-->' in line or line == '':
            continue
        texts.append(line)
    return texts

raw_texts = extract_texts_from_srt("demo.srt")
print(f"共提取 {len(raw_texts)} 条字幕文本")

共提取 17 条字幕文本


# combine srt parts into complete sentences

In [5]:
# def merge_sentences_by_model(text_chunks):
#     merged = []
#     system_prompt = "You are an expert at rewriting broken subtitles into full natural English sentences."
#     for idx, chunk in enumerate(text_chunks):
#         prompt = f"{system_prompt}\n\nPlease merge these fragments into complete sentences:\n" + "\n".join(chunk)
#         inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(model.device)
#         outputs = model.generate(**inputs, max_new_tokens=512, do_sample=False)
#         response = tokenizer.decode(outputs[0], skip_special_tokens=True)
#         # 去掉多余prompt
#         if "Please merge these fragments" in response:
#             response = response.split('Please merge these fragments into complete sentences:')[-1].strip()
#         merged.append(response)
#         print(f"完成第 {idx+1}/{len(text_chunks)} 批")
#     return merged

# print("begin combining")
# # 每组处理10条，防止太长爆显存
# chunks = [raw_texts[i:i+10] for i in range(0, len(raw_texts), 10)]

# merged_sentences = merge_sentences_by_model(chunks)

In [6]:
# [s.split('\n') for s in merged_sentences]

In [7]:
%pip install sentence-transformers scikit-learn nltk


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
import nltk
nltk.download('punkt')
print(nltk.data.path)

['/home/caesar/nltk_data', '/home/caesar/Documents/code-projects/routine-utilities/.venv/nltk_data', '/home/caesar/Documents/code-projects/routine-utilities/.venv/share/nltk_data', '/home/caesar/Documents/code-projects/routine-utilities/.venv/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data']


[nltk_data] Downloading package punkt to /home/caesar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
from nltk.tokenize import sent_tokenize
import nltk

nltk.data.path.append('/home/caesar/nltk_data')  # 如果你用的是默认下载路径，可以是 ~/.nltk_data

text = "This is sentence one. And this is sentence two."
sents = sent_tokenize(text)
print(sents)

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/home/caesar/nltk_data'
    - '/home/caesar/Documents/code-projects/routine-utilities/.venv/nltk_data'
    - '/home/caesar/Documents/code-projects/routine-utilities/.venv/share/nltk_data'
    - '/home/caesar/Documents/code-projects/routine-utilities/.venv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/home/caesar/nltk_data'
**********************************************************************


In [9]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
from nltk.tokenize import sent_tokenize
import numpy as np

# text = $json["text"]
text = raw_texts
sents = sent_tokenize(text)
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(sents)
clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=1.0)
labels = clustering.fit_predict(embeddings)

segments = []
current = []
last_label = labels[0]
for sent, label in zip(sents, labels):
    if label != last_label:
        segments.append(" ".join(current))
        current = []
    current.append(sent)
    last_label = label
if current:
    segments.append(" ".join(current))

print([{"method": "sbert_cluster", "segments": segments}])


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/home/caesar/nltk_data'
    - '/home/caesar/Documents/code-projects/routine-utilities/.venv/nltk_data'
    - '/home/caesar/Documents/code-projects/routine-utilities/.venv/share/nltk_data'
    - '/home/caesar/Documents/code-projects/routine-utilities/.venv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************
